In [1]:
import numpy as np
import os
import pickle
from pathlib import Path
import requests
import string
import re
from datasets import load_dataset

def decode(l):
    return ''.join([itos[i] for i in l])


# NLTK function to generate ngrams
import nltk
from nltk.util import ngrams
from nltk.probability import FreqDist

In [2]:
enron = load_dataset("snoop2head/enron_aeslc_emails", num_proc=8)

In [3]:
def process_enron_doc(text_list):
    processed_list = [None] * len(text_list)

    for i in range(0, len(text_list)):
        doc = text_list[i]
        #remove title 
        body = doc.partition("Body:")[2]
        #remove foreign characters, numbers, punctuation, line breaks, and go to lowercase
        processed_list[i] = re.sub(r'[^a-zA-z ]', ' ', body).translate(
            str.maketrans("","", string.punctuation)).replace(
            "\n", " ").lower()
    return processed_list

In [4]:
def avg_branching_factor(text, prev_word_dic={}, find_count=False):
    words = nltk.word_tokenize(text)
    word_dic = prev_word_dic
    for i,word in enumerate(words):
        if len(words) == i+1:
            continue
        if word_dic.get(word) == None:
            word_dic[word] = set([words[i+1]])
        else:
            w_set = word_dic[word]
            w_set.add(words[i+1])
    count = 0
    if find_count: 
        for k,v in word_dic.items():
            count += len(v)
        return count/len(word_dic)
    return word_dic

In [6]:
enron_processed_doc = process_enron_doc(enron['train']['text'])

In [7]:
enron_processed_doc[3]

'  randy   can you send me a schedule of the salary and level of everyone in the  scheduling group   plus your thoughts on any changes that need to be made     patti s for example  phillip'

In [9]:
for i in range(0, len(enron_processed_doc)): 
    if i == 0:
        prev_dic = avg_branching_factor(enron_processed_doc[i])
    elif i < len(enron_processed_doc) - 1:
        prev_dic = avg_branching_factor(enron_processed_doc[i], prev_word_dic = prev_dic)
    else:
        final_dic = avg_branching_factor(enron_processed_doc[i], prev_word_dic = prev_dic)
        for k,v in final_dic.items():
            count += len(v)
        result_enron = count/len(final_dic)
result_enron
#final dic is getting saved too 

NameError: name 'count' is not defined

In [13]:
count = 0
for k,v in final_dic.items():
        count += len(v)
result_enron = count/len(final_dic)
result_enron

10.67294048616932

In [14]:
len(final_dic.items()) #about 700 smaller than the vocab size 

596500